In [16]:
import glob
import os
import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [3]:
# Load the dataset from CSV file
data = pd.read_csv('data.csv')

In [4]:
data.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
0,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0
1,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0
2,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0
3,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0
4,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0


In [5]:
features = list(data.iloc[:, 0:192].columns)
X = data[features]
y = data['good_move']

In [7]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

print(categorical_columns) #ToDo:h8 missing
print(numerical_columns)

['a1', 'b1', 'c1', 'd1', 'e1', 'f1', 'g1', 'h1', 'a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2', 'a3', 'b3', 'c3', 'd3', 'e3', 'f3', 'g3', 'h3', 'a4', 'b4', 'c4', 'd4', 'e4', 'f4', 'g4', 'h4', 'a5', 'b5', 'c5', 'd5', 'e5', 'f5', 'g5', 'h5', 'a6', 'b6', 'c6', 'd6', 'e6', 'f6', 'g6', 'h6', 'a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7', 'a8', 'b8', 'c8', 'd8', 'e8', 'f8', 'g8']
['from_a1', 'from_b1', 'from_c1', 'from_d1', 'from_e1', 'from_f1', 'from_g1', 'from_h1', 'from_a2', 'from_b2', 'from_c2', 'from_d2', 'from_e2', 'from_f2', 'from_g2', 'from_h2', 'from_a3', 'from_b3', 'from_c3', 'from_d3', 'from_e3', 'from_f3', 'from_g3', 'from_h3', 'from_a4', 'from_b4', 'from_c4', 'from_d4', 'from_e4', 'from_f4', 'from_g4', 'from_h4', 'from_a5', 'from_b5', 'from_c5', 'from_d5', 'from_e5', 'from_f5', 'from_g5', 'from_h5', 'from_a6', 'from_b6', 'from_c6', 'from_d6', 'from_e6', 'from_f6', 'from_g6', 'from_h6', 'from_a7', 'from_b7', 'from_c7', 'from_d7', 'from_e7', 'from_f7', 'from_g7', 'from_h7', 'fro

In [8]:
for feature_name in categorical_columns:
  vocabulary = X[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [9]:
# input function
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

In [12]:
# split data into batches

def split_into_batches(df, batch_size=100000):
  nb_rows = len(df.index)
  intervals = []
  
  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)
  
  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)
  
  batches_X = []
  batches_y = []
  
  for i in range(0, len(intervals) - 1):
    batches_X.append(data.iloc[intervals[i]:intervals[i + 1], :][features])
    batches_y.append(data.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return batches_X, batches_y

In [13]:
batches_X, batches_y = split_into_batches(data)

In [14]:
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Dell\\AppData\\Local\\Temp\\tmppi9ghga6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_ta

In [15]:
# train
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
  input_functions.append(make_input_fn(df_X, df_y))

len(input_functions)

17

In [17]:
# train the model on all the input functions
i = 1
for input_function in input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)

<======================================== NEW BATCH ========================================>
Batch: 1
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners bef

In [18]:
# Save the trained model
with open('model.pkl', 'wb') as f:
    pickle.dump(linear_est, f)

AttributeError: Can't pickle local object 'LinearClassifierV2.__init__.<locals>._model_fn'

In [19]:
# save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = './'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['s

In [20]:
print(feature_columns)

[VocabularyListCategoricalColumn(key='a1', vocabulary_list=('R', 'None', 'K', 'r', 'B', 'q', 'Q', 'b', 'n', 'N'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='b1', vocabulary_list=('N', 'None', 'R', 'K', 'r', 'B', 'b', 'Q', 'q', 'k', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='c1', vocabulary_list=('B', 'None', 'R', 'r', 'K', 'N', 'Q', 'b', 'q', 'n', 'k'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='d1', vocabulary_list=('Q', 'None', 'R', 'B', 'K', 'n', 'N', 'q', 'r', 'b', 'k'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='e1', vocabulary_list=('K', 'None', 'R', 'r', 'B', 'k', 'N', 'q', 'Q', 'b', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='f1', vocabulary_list=('B', 'None', 'R', 'Q', 'r', 'K', 'q', 'N', 'k', 'b', 'n'), dtype=tf.string, defa